<a href="https://colab.research.google.com/github/dolmani38/Summary/blob/master/real_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Beginners Guide to Text Generation using LSTMs
https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms

In [1]:
# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

# set seeds for reproducability
from tensorflow.random import set_seed
from numpy.random import seed
set_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

Text dataset을 만들기 위해 위키백과에서 스크랩핑...

In [2]:
!pip install wikipedia
import wikipedia as wiki
wiki.set_lang('ko')

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=47da6791ac7576c2f87c5490c9aca14148a4c1f8add38645debe0017f7e41220
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [3]:
def __search_from_wiki(question,max_rank):
  results = wiki.search(question,results=max_rank)
  print(results)
  contents = []
  for result in results:
    try:
      page = wiki.page(result)
      #print(f"Top wiki result: {page}")
      text = page.content
      ln = len(text)
      print(ln)
      if ln < 4000:
        contents.append(text)
      else:
        contents.append(text[0:4000])
    except Exception as ex:
      print(ex)
  return contents


all_headlines = __search_from_wiki("전래동화", 100)

len(all_headlines)

['동화', '신 전래동화', '꾸러기 수비대', '호시조라 미유키', '아동 문학', '한국의 사찰', '해와 달이 된 오누이', '거북', '잠자는 숲속의 미녀', '이한갈', '정은찬', '김기두 (배우)', '옛날 옛적에 (애니메이션)', '계룡선녀전 (드라마)', '동요', '이상훈 (1976년)', '최홍일', '육진수 (격투기 선수)', '제비', '장석현 (연예인)', '유다미', '박재훈 (배우)', '최지웅', '밀교 (불교)', '한다은', '재희', '정미남', '안젤리나 다닐로바', '도깨비', '기탄교육', '국지용', '의왕백운호수축제', '박민경', '정정아', '콩딱쿵! 이야기 주머니', '윤기원 (배우)', '선녀와 나무꾼', '남생이', '대장화홍련전', '콩쥐팥쥐 (동음이의)', '토끼전', '미녀와 야수', '지대한', '이설구', '콩쥐팥쥐전', '이서휴게소', '은비까비의 옛날옛적에', '빨간 자전거', '김덕현 (배우)', '홍석연', '아시리아인', '티베트', '백조의 호수', '도교', '서예', '켈트 다신교', '금도끼 은도끼', '장화, 홍련 (동음이의)', '제네시스 (밴드)', '허구 국가', '노상현', '타이완의 문화', '도깨비 (동음이의)', '골디락스', '선녀강림', '자와어', '손춘익', '안동국제탈춤페스티벌', '윌리엄 버틀러 예이츠', '성덕대왕신종', '고려', '모모타로', '한국 문학', '라푼젤 (영화)', '토비트', '홍석천', '조선 후기의 문학', '송월동 동화마을', '계룡선녀전', '이집트', '응우옌 왕조', '아시아의 역사', '안지환', '외래어', '메이플 월드', '프랑스인', '스키타이족', '인도네시아', '이탈리아', '네버랜드', 'MBC 창작동요제', '프랑크인', '일본 제국', '김환영 (작가)', '드랑 나흐 오스텐', '조선', '바이킹', '오윤 (화가)', '원나라', '앱북']
685
156
1554
1601
678
885
392
1377

97

In [4]:
all_headlines[3]

'호시조라 미유키(星空 (ほしぞら) みゆき 한국명:김다솜)는 토에이 애니메이션 제작의 애니메이션《스마일 프리큐어!》에 등장하는 가공의 인물이다. 애니메이션에서 목소리를 연기한 성우는 일본판은 후쿠엔 미사토, 한국판은 양정화이다.\n\n\n== 개요 ==\n프리큐어 시리즈 9번째 작품에 등장하는 7대째 프리큐어 전사 큐어 해피로 변신하는 인물. 가족은 부모님이 계시며, 외동딸이다. 머리 모양은 양갈래를 코로네와 소라빵모양처럼 작게 돌돌 말아 노란색 리본으로 묶고 있다.\n그림책과 동화책 읽는 것을 가장 좋아하며, 모든지 \'해피\'한게 좋다는 항상 긍정적이고 해맑은 여중생이다. 첫사랑 상대는 피터팬과 백마 탄 왕자이며, 가장 좋아하는 책은 안데르센의 동화 신데렐라. 기분이 좋아 흥분하면 "울트라 해피!(일본어: ウルトラハッピー!)"라고 말하는 말버릇이 있다. 반면 조금 맘에 안들거나 할때는 입을 삐죽 내밀며 "합뿌뿌~(일본어: ハップップー)"라고 하기도 한다.\n공부도 운동도 별로 못하고 덜렁이에 실수가 많지만, 결코 실망하는 법이 없이 한결같이 앞으로 전진하는 명랑하고 발랄한 성격인 소녀. 겉보기에는 미덥지 못하지만, 항상 모두를 행복하게 하기 위해 최선을 다하며 그 미소를 빼앗으려는 악당들에게도 과감히 맞서는 강하고 상냥한 마음은 친구들과 주변 사람들을 매료시키고있다. 한편 의외로 맘이 여리고 섬세해서 상처받기 쉬운 면도 있지만 눈물 보이는 것을 싫어하며 속으로 꾹꾹 누르며 항상 웃는 얼굴을 보여준다.\n어릴 적에 순수하고 밝은 성격과 달리 친구만들기가 어려운 부끄럼많고 내성적인 성격이였으나, 예전에 할머니집에서 살았던 마을에서 만난 숲의 소녀 덕분에 자신감과 용기를 얻는 계기가 되었다.\n다른 바보 3명과 달리 모든 과목을 못하는 것 같다. 16화에서는 다 아는 속담을 전래동화 드립을 쳤다.\n\n\n== 큐어 해피 ==\n성스러운 빛의 힘을 가진 프리큐어로서 변신한 모습. 이미지 색상은 분홍색.\n\n변신 주문은 "프리큐어 스마일 차지!"\n변신 후 대사

In [5]:
def clean_text(txt):
    txt = txt.replace('\n','')
    return txt 

corpus = [clean_text(x) for x in all_headlines]
corpus[:10]

['동화(童話)란 어린이를 독자층으로 하는 이야기를 말한다. 아동 문학의 한 장르이다.판타지나 우화, 프로파간다와는 구분할 필요가 있다.== 나라별 동화 ==그리스의 이솝우화, 이탈리아의 피노키오, 독일의 헨젤과 그레텔 등 동화로 알려진 문학이 있다.=== 한국 ===한국에서는 흔히 어린이를 위한 옛날 이야기를 동화라고 이해하고 있다. 이와 같은 해석은 이 문학 유형이 갖고 있는 고유한 특성을 간접적으로 설명한다고 볼 수 있다. 왜냐하면 전통적 의미에서 동화는 한 지방이나 국가에서 세대를 거치면서 자기 나라말로 구전된 것에서 그 뿌리를 두고 있기 때문이다. 이러한 전통적 동화는 종류가 대단히 한정될 수밖에 없다. 현대에 들어서 동화의 창작은 간과할 수 없는 문학 작업으로 평가되고 있다.한국의 본격적인 창작동화는 1923년 마해송(馬海松)의 〈바위나리와 아기별〉이 발표되고, 이어서 《어린이》·《아이생활》·《별나라》 등 여러 잡지와 구연회(口演會)를 통하여 방정환·고한승(高漢承)·진장섭(秦長燮)·정인섭·이정호(李定鎬) 등 색동회 동인들과 기타 이주홍(李周洪) 등의 작가들이 창작활동을 활발히 전개함으로써 비로소 기반을 쌓게 되었다. 현재도 보리출판사, 사계절출판사, 예영커뮤니케이션 등 다수의 출판사에서 어린이들을 위한 창작동화들을 출판하고 있다.== 나라별 동화 작가 ==== 각주 ==',
 '《신 전래동화》는 2018년에 개봉한 대한민국의 영화이다.== 캐스팅 ==== 같이 보기 ==2018년 대한민국의 영화 목록== 외부 링크 ==신 전래동화 - 네이버 영화신 전래동화 - 다음 영화신 전래동화 - 무비스트신 전래동화 - 한국영화 데이터베이스',
 '《꾸러기 수비대》 (혹은《출동! 12레인저》)는 십이지를 소재로 한 애니메이션이다.일본에서는 《십이전지 폭렬 에토레인저》(十二戦支 爆烈エトレンジャー)라는 이름으로 NHK 위성 제2텔레비전을 통해 1995년에 방영되었다.== 개요 ==한국어 더빙판으로는 1996년 KBS 2TV에서 방영한 《꾸러기 수비대》와 투니버스에서

In [10]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]

[[226, 6082],
 [226, 6082, 1385],
 [226, 6082, 1385, 1042],
 [226, 6082, 1385, 1042, 6083],
 [226, 6082, 1385, 1042, 6083, 67],
 [226, 6082, 1385, 1042, 6083, 67, 198],
 [226, 6082, 1385, 1042, 6083, 67, 198, 227],
 [226, 6082, 1385, 1042, 6083, 67, 198, 227, 6084],
 [226, 6082, 1385, 1042, 6083, 67, 198, 227, 6084, 417],
 [226, 6082, 1385, 1042, 6083, 67, 198, 227, 6084, 417, 11]]

In [11]:
len(inp_sequences)

47930

In [12]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [14]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1035, 10)          250810    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               44400     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 25081)             2533181   
Total params: 2,828,391
Trainable params: 2,828,391
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(predictors, label, epochs=10, verbose=5)

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


In [28]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [25]:
print (generate_text("b", 50, model, max_sequence_len))
print (generate_text("c", 30, model, max_sequence_len))

B 토빗기 영어 지은이와 영어 Children'S 영어 Book 영어 Penmanship 또는 Calligraphy 은 붓으로 글씨를 또는 일컫는다 사는 작품에 왕조이다 이불란사와 초문사이다 문화관광부에 등록된 사찰의 개수는 926개이다 삼보사찰 삼보 三寶 등이 있다 귀하게 1986년 7월 〈퐁당퐁당〉이나 〈커다란 꿀밤 2008년 1991년 1991년 예능 역 예능 1994년 Kbs 연기대상 스페셜 Sbs 연기대상 공채
C 聖德大王神鍾 는 러시아의 차이콥스키가 작곡한 발레 음악이자 또는 완조 열리는 열리는 9월에 타로가 Rose · 복서 · 무축 · 무축 · 무축 · 무축 · 무축 · 무축 · 무축


In [29]:
print (generate_text("이탈리아", 30, model, max_sequence_len))

이탈리아 영어 Vikings 노르웨이어 Vikinger Vikinger Vikinger Vikinger 스웨덴어 뉘노르스크 뉘노르스크 Vikingar 아이슬란드어 Víkingar 은 노르드어 비킹 는 노르드어 Vikingr 이 골디락스 표현이다 북게르만족 노르드인이고 노르드어를 노르드어를 지역의 구전된 한다 이들은


In [2]:
document = """
주호영 국민의힘 원내대표는 22일 고위공직자범죄수사처(공수처)법 개정과 가덕도 신공항 건설 등을 밀어붙이고 있는 문재인 정권과 더불어민주당을 향해 "이제 끝이 보인다"며 "짓밟힌 풀들이 아우성 치는 국민적 저항에 직면할 것"이라고 경고했다.
주 원내대표는 이날 자신의 페이스북에 "문재인 정권이 공수처법 개정을 위한 '군사작전'에 돌입하겠다고 엄포를 놓고 있다"며 "정의당을 끌어들이기 위해 꼼수 선거법에 묶어 '패스트트랙'이라는 불법·탈법으로 만들어낸 공수처법을 시행도 해보지 않고 고치려 하는 것"이라고 지적했다.
이어 주 원내대표는 "야당 원내대표인 제게 문재인 대통령은 사람 좋아보이는 표정으로 '공수처는 야당의 동의 없이는 절대 출범할 수 없는 것'이라고 얘기했고, 야당이 유엔 안보리 상임이사국처럼 공수처장 임명에 '비토권'을 행사할 수 있는데 무얼 걱정하느냐고, 여당 사람들이 우리를 속였다"며 "거짓말이라는 비난을 개의치 않는 사람들"이라고 꼬집었다.
주 원내대표는 "이해찬 전 민주당 대표가 얘기한 '민주당 20년 집권'의 토대가 올해 안에 완성된다"며 "탈원전과 동남권 신공항은 문 대통령이 대선 공약으로 내건 사업이니 여기에 불법이 있었다고 시비를 거는 것은 민주주의를 부정하는 것이라고 청와대 출신 윤건영 민주당 의원이 윽박지른다. 이제 '민주주의 없는 민주당'이 법위에 군림하는 '반민주'를 거리낌없이 획책하는 것"이라고 언급했다.
그러면서 주 원내대표는 "표를 얻기 위해 나라 곳간을 다 허물어뜨렸고, 재정 운용에서 신중함은 사라졌다"며 "괴물 공수처가 출범하면 공무원 누구나 대통령과 권력이 지시하는 범죄행위에 거리낌 없이 가담할 것이다. 청와대와 권부 요직에 앉아 불법으로 각종 이권을 챙기는 권력자들에 대한 사건이 불거져도 공수처가 사건을 가져가 버리면 그만"이라고 우려했다.
주 원내대표는 "문 대통령은 제게 '공수처는 고위 공직자들을 처벌하는 것인데 왜 야당이 반대하는지 이해할 수 없다'고 했는데, 그런 분이 청와대와 대통령 주변을 감시하는 특별감찰관은 취임 이후 지금까지 왜 임명하지 않았는가"라며 "공수처는 권력형 비리의 쓰레기 하치장, 종말 처리장이 될 것"이라고 비판했다.
문재인 정부를 향해 주 원내대표는 "문 대통령과 그 사도들은 법치가 미치지 않는 무오류의 화신이 될 것"이라며 "오류를 인정하지 않는 존재가 바로 신이며 그 아래에는 자신들의 지도자를 목숨바쳐 지킴으로서 정의를 실현하겠다는 추종자들로 넘쳐 난다. 공수처는 지도자의 신성을 인정하지 않는 세력을 정죄하는 수단으로 전락할 것"이라고 질타했다.
주 원내대표는 "저도 법조인이지만 대통령과 공수처장이 마음대로 검사들과 수사관들을 임명하는 이 끔찍한 사법기구가 어떤 일을 할지 두렵기만 하다"며 "공수처는 검찰과 경찰 위에 있는 사법기구로, 헌법과 법으로 독립성을 보장하는 검찰총장을 이렇게 핍박하는 정권이 공수처를 어떻게 운영할지 불을 보듯 뻔한 일"이라고 예측했다.
그러면서 주 원내대표는 "추미애 법무장관을 앞장 세워 윤석열 검찰의 권력 비리 수사를 저지하려다가 난관에 봉착하자 무슨 수를 써서라도 공수처를 출범시키려 한다. 공수처장 자리에는 추미애보다 더 한 막무가내 내 편을 앉힐 게 분명한 것"이라며 "문 정권의 파렴치와 오만함을 최전선에서 온 몸으로 겪어온 저로서는 민주당이 내일부터 국회에서 보일 행태가 환히 보인다. 180석의 민주당이 또 군사작전을 개시하면 그걸 누가 막겠는가"라고 성토했다.
주 원내대표는 "공수처법을 막을 힘이 우리 야당에게는 없다. 삭발하고 장외투쟁해 봐야 눈 하나 깜짝할 사람들이 아닌 것"이라며 "대란대치(大亂大治), 세상을 온통 혼돈 속으로 밀어넣고 그걸 권력 유지에 이용한다는 게 이 정권의 통치기술"이라고 규탄했다.
아울러 주 원내대표는 "권력은 바람, 국민은 풀이다. 바람이 불면 청보리 밭의 보리가 눕는다"며 "권력은 풀들이 다시는 일어서지 못하도록 풀을 짓밟지만 풀들은 다시 일어난다. 시인 김수영은 '바람보다 먼저 눕지만, 바람보다 먼저 일어나는' 민초의 힘을 노래했다"고 말했다.
마지막으로 주 원내대표는 "문재인 정권은 이제 곧 국회에서 광장에서 짓밟힌 풀들이 일어서서 아우성치는 모습을 지켜보게 될 것"이라며 "대란대치를 끝장내려는 국민적 저항에 직면할 것"이라고 거듭 강조했다.
"""

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([document])
total_words = len(tokenizer.word_index) + 1

In [4]:
print(total_words)

395


In [5]:
for word,index in tokenizer.word_index.items():
  print(index,word)

1 원내대표는
2 주
3 것
4 이라고
5 며
6 문재인
7 않는
8 문
9 이라며
10 이제
11 풀들이
12 수
13 대통령과
14 공수처는
15 될
16 있는
17 향해
18 보인다
19 짓밟힌
20 국민적
21 저항에
22 직면할
23 정권이
24 위해
25 공수처법을
26 제게
27 대통령은
28 '공수처는
29 없는
30 야당이
31 공수처장
32 사람들이
33 민주당
34 그러면서
35 공수처가
36 청와대와
37 왜
38 그
39 인정하지
40 이
41 공수처를
42 권력
43 게
44 정권의
45 민주당이
46 국회에서
47 그걸
48 권력은
49 먼저
50 주호영
51 국민의힘
52 22일
53 고위공직자범죄수사처
54 공수처
55 법
56 개정과
57 가덕도
58 신공항
59 건설
60 등을
61 밀어붙이고
62 정권과
63 더불어민주당을
64 끝이
65 아우성
66 치는
67 경고했다
68 이날
69 자신의
70 페이스북에
71 공수처법
72 개정을
73 위한
74 '군사작전'에
75 돌입하겠다고
76 엄포를
77 놓고
78 있다
79 정의당을
80 끌어들이기
81 꼼수
82 선거법에
83 묶어
84 '패스트트랙'이라는
85 불법·탈법으로
86 만들어낸
87 시행도
88 해보지
89 않고
90 고치려
91 하는
92 지적했다
93 이어
94 야당
95 원내대표인
96 사람
97 좋아보이는
98 표정으로
99 야당의
100 동의
101 없이는
102 절대
103 출범할
104 것'이라고
105 얘기했고
106 유엔
107 안보리
108 상임이사국처럼
109 임명에
110 '비토권'을
111 행사할
112 있는데
113 무얼
114 걱정하느냐고
115 여당
116 우리를
117 속였다
118 거짓말이라는
119 비난을
120 개의치
121 사람들
122 꼬집었다
123 이해찬
124 전
125 대표가
126 얘기한
127 '민주당
128 20년
129 집권'의
130 토대가
131 올해
132 안에
133 완성된다
134 탈원전과
135 동남

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import (Dense, 
                                     BatchNormalization, 
                                     LeakyReLU,
                                     Softmax,
                                     Reshape, 
                                     Conv2DTranspose,
                                     Conv2D,
                                     Dropout,
                                     Flatten)
import matplotlib.pyplot as plt

In [7]:

def make_generator_model():
    model = tf.keras.Sequential()
    model.add(Dense(256, use_bias=False, input_shape=(50,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    #model.add(Reshape((256,)))
    assert model.output_shape == (None,256) # Note: None is the batch size

    model.add(Dense(64, use_bias=False))
    model.add(BatchNormalization())
    model.add(Softmax())

    #model.add(Reshape((64,)))
    assert model.output_shape == (None,64) # Note: None is the batch size

    model.summary()
    return model

generator = make_generator_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               12800     
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16384     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
softmax (Softmax)            (None, 64)                0         
Total params: 30,464
Trainable params: 29,824
Non-trainable params: 640
__________________________________________________

In [25]:
# Create a random noise and generate a sample
noise = tf.random.normal([10, 50])
generated_text = generator(noise, training=False)


In [26]:
generated_text

<tf.Tensor: shape=(10, 64), dtype=float32, numpy=
array([[0.01513626, 0.01168383, 0.00938422, 0.01528631, 0.00754556,
        0.01316108, 0.00716574, 0.01651304, 0.02208476, 0.01300555,
        0.01769161, 0.02049619, 0.01499179, 0.00776885, 0.01241255,
        0.00439605, 0.01365743, 0.02238177, 0.01237988, 0.01307812,
        0.02075309, 0.01090393, 0.01609082, 0.01354598, 0.01462952,
        0.03287378, 0.01270114, 0.02632295, 0.01381229, 0.01298717,
        0.01155249, 0.01093738, 0.01095644, 0.02230717, 0.00636088,
        0.01055169, 0.02025925, 0.01624437, 0.01670043, 0.01802511,
        0.00973136, 0.01906495, 0.02147042, 0.02702578, 0.01335141,
        0.02191122, 0.01410525, 0.01707666, 0.01268106, 0.03204919,
        0.00859776, 0.00969196, 0.02576758, 0.01689919, 0.01780741,
        0.00386661, 0.01315148, 0.00807407, 0.01212006, 0.02364339,
        0.02022686, 0.02732211, 0.02484374, 0.01078396],
       [0.01546958, 0.01436912, 0.01060897, 0.02074404, 0.01362762,
        0

In [27]:
generated_text2 = generated_text[0] * total_words
generated_text2

<tf.Tensor: shape=(64,), dtype=float32, numpy=
array([ 5.9788213,  4.6151137,  3.7067683,  6.038094 ,  2.9804945,
        5.198628 ,  2.8304672,  6.5226507,  8.723481 ,  5.137191 ,
        6.9881845,  8.095993 ,  5.921757 ,  3.0686965,  4.9029584,
        1.7364391,  5.394686 ,  8.840798 ,  4.8900537,  5.165857 ,
        8.197471 ,  4.3070526,  6.355872 ,  5.350662 ,  5.778662 ,
       12.985143 ,  5.0169487, 10.397564 ,  5.455855 ,  5.1299314,
        4.563235 ,  4.3202634,  4.3277955,  8.811333 ,  2.5125494,
        4.1679173,  8.002402 ,  6.416525 ,  6.5966706,  7.119917 ,
        3.843886 ,  7.5306535,  8.480815 , 10.675183 ,  5.273806 ,
        8.654934 ,  5.5715747,  6.7452807,  5.0090194, 12.659431 ,
        3.396117 ,  3.8283262, 10.178193 ,  6.675179 ,  7.033929 ,
        1.5273108,  5.1948333,  3.1892574,  4.787424 ,  9.33914  ,
        7.9896083, 10.7922325,  9.813275 ,  4.2596645], dtype=float32)>

In [28]:
generated_sum = ""
for idx in generated_text2:
  #print(int(idx))
  for word,index in tokenizer.word_index.items():
      if index == int(idx):
          generated_sum += word + ' '
          break

print(generated_sum)

며 이라고 것 문재인 주 며 주 문재인 문 며 문재인 문 며 것 이라고 원내대표는 며 문 이라고 며 문 이라고 문재인 며 며 수 며 이제 며 며 이라고 이라고 이라고 문 주 이라고 문 문재인 문재인 않는 것 않는 문 이제 며 문 며 문재인 며 수 것 것 이제 문재인 않는 원내대표는 며 것 이라고 이라며 않는 이제 이라며 이라고 


In [16]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(Dense(256, use_bias=False, input_shape=(1024,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    #model.add(Reshape((256,)))
    assert model.output_shape == (None,256) # Note: None is the batch size

    model.add(Dense(64, use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())    
    #model.add(Reshape((64,)))
    assert model.output_shape == (None,64) # Note: None is the batch size

    model.add(Dense(32, use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())    
    #model.add(Reshape((64,)))
    assert model.output_shape == (None,32) # Note: None is the batch size

    model.add(Dense(1))
    #model.add(Softmax())    
    model.summary()
    return model

discriminator = make_discriminator_model()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 256)               262144    
_________________________________________________________________
batch_normalization_5 (Batch (None, 256)               1024      
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                16384     
_________________________________________________________________
batch_normalization_6 (Batch (None, 64)                256       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 32)               

In [13]:
!pip install sentence-transformers

In [14]:
from sentence_transformers import SentenceTransformer
# embedder download...
embedder = SentenceTransformer('xlm-r-large-en-ko-nli-ststb')

In [35]:
#generated_encode = embedder.encode([generated_sum])
decision = discriminator(generated_embedding)
print (decision)

tf.Tensor(
[[ 0.02082823]
 [-0.05240061]
 [ 0.07198735]
 [ 0.01703282]
 [ 0.4902227 ]
 [ 0.00279847]
 [ 0.27390167]
 [ 0.4966565 ]
 [ 0.53378   ]
 [ 0.14577103]
 [ 0.00360338]
 [-0.20994215]
 [ 0.09074389]
 [ 0.1196237 ]
 [ 0.5612309 ]
 [ 0.09767302]
 [ 0.25924385]
 [ 0.22576228]
 [-0.07519284]
 [-0.00087412]
 [ 0.377385  ]
 [ 0.2900403 ]
 [ 0.06445952]
 [ 0.22728139]
 [ 0.23552147]
 [ 0.10944294]
 [ 0.16658542]
 [ 0.39936018]
 [ 0.13387027]
 [-0.29398772]
 [ 0.2340291 ]
 [ 0.4496358 ]], shape=(32, 1), dtype=float32)


In [18]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [30]:
BATCH_SIZE = 32
EPOCHS = 60
noise_dim = 50
seed = tf.random.normal([1, noise_dim])

In [33]:
def generated_code_to_text(generated_code,total_words):
  generated_sum = []
  generated_code = [c * total_words for c in generated_code]
  for g in generated_code:
    summary_txt = ""
    for idx in g:
      #print(int(idx))
      for word,index in tokenizer.word_index.items():
          if index == int(idx):
              summary_txt += word + ' '
              break
    generated_sum.append(summary_txt)
  return generated_sum, embedder.encode(generated_sum)

In [34]:
noise = tf.random.normal([BATCH_SIZE, 50])
generated_code = generator(noise, training=False)
generated_text,generated_embedding = generated_code_to_text(generated_code,total_words)
print(generated_text[0])

이제 문재인 주 문재인 문재인 것 주 않는 며 공수처는 것 것 며 며 것 이제 며 문재인 문 대통령과 않는 이라고 며 풀들이 며 이라고 이라고 문 것 문재인 않는 며 주 문 것 이라며 문재인 않는 문 이라며 것 것 것 문 이라고 주 이라고 문재인 며 며 풀들이 며 문재인 이제 이라고 주 것 주 않는 이라고 주 않는 않는 것 


In [36]:
@tf.function
def train_step(images):
  
    # 1 - Create a random noise to feed it into the model
    # for the text generation
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    
    # 2 - Generate text and calculate loss values
    # GradientTape method records operations for automatic differentiation.
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_code = generator(noise, training=True)
      generated_text,generated_embedding = generated_code_to_text(generated_code,total_words)
      real_embedding = embedder.encode(images)
      real_output = discriminator(real_embedding, training=True)
      fake_output = discriminator(generated_embedding, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    # 3 - Calculate gradients using loss values and model variables
    # "gradient" method computes the gradient using 
    # operations recorded in context of this tape (gen_tape and disc_tape).
    
    # It accepts a target (e.g., gen_loss) variable and 
    # a source variable (e.g.,generator.trainable_variables)
    # target --> a list or nested structure of Tensors or Variables to be differentiated.
    # source --> a list or nested structure of Tensors or Variables.
    # target will be differentiated against elements in sources.

    # "gradient" method returns a list or nested structure of Tensors  
    # (or IndexedSlices, or None), one for each element in sources. 
    # Returned structure is the same as the structure of sources.
    gradients_of_generator = gen_tape.gradient(gen_loss, 
                                               generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, 
                                                discriminator.trainable_variables)
    
    # 4 - Process  Gradients and Run the Optimizer
    # "apply_gradients" method processes aggregated gradients. 
    # ex: optimizer.apply_gradients(zip(grads, vars))
    """
    Example use of apply_gradients:
    grads = tape.gradient(loss, vars)
    grads = tf.distribute.get_replica_context().all_reduce('sum', grads)
    # Processing aggregated gradients.
    optimizer.apply_gradients(zip(grads, vars), experimental_aggregate_gradients=False)
    """
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [37]:
dataset = []
for i in range(1000):
  dataset.append(document)

In [38]:
import time
from IPython import display # A command shell for interactive computing in Python.

def train(dataset, epochs):
  # A. For each epoch, do the following:
  for epoch in range(epochs):
    start = time.time()
    # 1 - For each batch of the epoch, 
    for image_batch in dataset:
      # 1.a - run the custom "train_step" function
      # we just declared above
      train_step(image_batch)

    # 4 - Print out the completed epoch no. and the time spent
    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [39]:
train(dataset, EPOCHS)

OperatorNotAllowedInGraphError: ignored